# Path

In [1]:
import sys
import os

project_path = r'/home/craxiss/Documents/projects/kapsul_etkinlik_v2'

os.chdir(project_path)
sys.path.append(project_path)

# Imports

In [2]:
from helpers import column_formatter, utf8_convert
from config import data_path as path
from json import dump
import pandas as pd
import numpy as np

# Read

In [3]:
appeals = pd.read_excel(path+'/1-raw/basvuru.xlsx')

# Format Numbers

In [4]:
for char in ['(', ' ', ')', '-', '+']:
    appeals['Cep Telefonunuz'] = appeals['Cep Telefonunuz'].str.replace(char, '', regex=True)

appeals['Cep Telefonunuz'] = appeals['Cep Telefonunuz'].str[-10:]
appeals['Cep Telefonunuz'] = appeals['Cep Telefonunuz'] + 'str'

# Telefon numaralarında olan gereksiz karakterleri silip sondan 10 karakteri alıp sonuna 'str' ekledim.
# str eklememin sebebi pandas kütüphanesinin otomatik olarak telefon numaralarını tam sayıya çevirmesi.

# Format Names

In [5]:
appeals['İsim - Soy isim'] = column_formatter(appeals['İsim - Soy isim'])
# column_formatter isimleri küçültüp birleştiriyor ve türkçe karakterleri siliyor.

# Get Appealed Lessons

In [6]:
lesson_user_dict = dict()
lessons_list = list()

for col in appeals.columns[4:]:
    appeals[col] = appeals[col].str.split(',')

for i in range(appeals.shape[0]):
    row = appeals.iloc[i:i+1]
    name = utf8_convert(row['İsim - Soy isim'].iloc[0])
    
    lessons_list_temp = list()
    for col in appeals.columns[4:]:
        lessons = row[col].iloc[0]

        if type(lessons) == type(float('NaN')) or type(lessons) == type(np.nan):
            lessons = []

        if name not in lesson_user_dict:
            lesson_user_dict[name] = lessons
        else:
            lesson_user_dict[name].extend(lessons)

        lessons_list_temp.extend(lessons)
    lessons_list.append(','.join(lessons_list_temp))    

appeals['Başvurulan Dersler'] = lessons_list

In [7]:
all = list()
for lessons in appeals['Başvurulan Dersler']:
    all.extend(list(map(lambda x: x.strip(), lessons.split(','))))

kw = dict(zip(set(all), list(map(lambda x: x+" 101" if("101" not in x and x != '') else x, set(all)))))

lesson_names = pd.DataFrame([kw])

appeals = appeals[['İsim - Soy isim', 'Cep Telefonunuz', 'E-mail Adresiniz', 'Zaman damgası', 'Başvurulan Dersler']]
appeals.columns = ['name', 'phone', 'mail', 'time', 'lessons']

# Charset Normalization

In [9]:
for col in appeals.columns[:-1]:
    for i in range(appeals[col].shape[0]):
        appeals.loc[i,col] = utf8_convert(appeals.loc[i,col])

# Write

## XLSX

In [10]:
with pd.ExcelWriter(path+'/2-cleaned/basvuru.xlsx') as writer:
    appeals.to_excel(writer, sheet_name='Başvurular')
    lesson_names.T.to_excel(writer, sheet_name='Dersler')